# DS 320 Final Project - Thomas

In [85]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from tabulate import tabulate
from sklearn import metrics


In [2]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)


Mounted at /content/gdrive/


In [3]:
plays = pd.read_csv('/content/gdrive/MyDrive/plays.csv')
week1 = pd.read_csv('/content/gdrive/MyDrive/week1.csv')

In [4]:
week2 = pd.read_csv('/content/gdrive/MyDrive/football/week2.csv')
week3 = pd.read_csv('/content/gdrive/MyDrive/football/week3.csv')

In [5]:
week4 = pd.read_csv('/content/gdrive/MyDrive/football/week4.csv')
week5 = pd.read_csv('/content/gdrive/MyDrive/football/week5.csv')

In [6]:
week6 = pd.read_csv('/content/gdrive/MyDrive/football/week6.csv')
week7 = pd.read_csv('/content/gdrive/MyDrive/football/week7.csv')

In [7]:
week8 = pd.read_csv('/content/gdrive/MyDrive/football/week8.csv')
week9 = pd.read_csv('/content/gdrive/MyDrive/football/week9.csv')

In [8]:
week10 = pd.read_csv('/content/gdrive/MyDrive/football/week10.csv')
week11 = pd.read_csv('/content/gdrive/MyDrive/football/week11.csv')

In [9]:
week12 = pd.read_csv('/content/gdrive/MyDrive/football/week12.csv')
week13 = pd.read_csv('/content/gdrive/MyDrive/football/week13.csv')

In [10]:
week14 = pd.read_csv('/content/gdrive/MyDrive/football/week14.csv')
week15 = pd.read_csv('/content/gdrive/MyDrive/football/week15.csv')

In [11]:
week16 = pd.read_csv('/content/gdrive/MyDrive/football/week16.csv')
week17 = pd.read_csv('/content/gdrive/MyDrive/football/week17.csv')

In [12]:
df = plays.copy()
words = "INTERCEPTED|PENALTY|FUMBLES"
passes = df[df['playDescription'].str.contains(" pass ", case=False, na=False)]
filtered = passes[~passes['playDescription'].str.contains(words, case=False, na=False)]
filtered.shape[0]

16197

In [13]:
short = filtered[filtered['playDescription'].str.contains("short", case=False, na=False)]
deep = filtered[filtered['playDescription'].str.contains("deep", case=False, na=False)]

In [41]:
df = pd.concat([week1, week2, week3, week4, week5, week6, week7, week8, week9, week10, week11, week12, week13, week14, week15, week16, week17])

In [42]:
df = df[df["position"] == "WR"] 

In [43]:
df = df[df['event'] == 'pass_arrived']

In [45]:
df = df[['a', 's', 'dis', 'gameId', 'playId']]

In [51]:
short = short[['gameId', 'playId', 'passResult']]

In [52]:
deep = deep[['gameId', 'playId', 'passResult']]

In [53]:
df_long = df.merge(deep, how = "inner")

In [54]:
df_short = df.merge(short, how = "inner")

In [77]:
X = df_short[['a', 's', 'dis']]
y = df_short['passResult']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [78]:
short_clf=RandomForestClassifier(n_estimators=100)

short_clf.fit(X_train,y_train)
y_pred=short_clf.predict(X_test)


In [80]:
rfc_short_acc = metrics.accuracy_score(y_test, y_pred)

print("Model accuracy on short passes:",metrics.accuracy_score(y_test, y_pred))

Model accuracy on short passes: 0.7878366823428816


In [81]:
X = df_long[['a', 's', 'dis']]
y = df_long['passResult']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [82]:
long_clf=RandomForestClassifier(n_estimators=100)

long_clf.fit(X_train,y_train)
y_pred=long_clf.predict(X_test)

In [84]:
print("Model accuracy on long passes:",metrics.accuracy_score(y_test, y_pred))
rfc_deep_acc = metrics.accuracy_score(y_test, y_pred)

Model accuracy on long passes: 0.573408947700063


In [66]:
df_short['binaryPassResult'] = [0 if x == 'I' else 1 for x in df_short["passResult"]]
df_long['binaryPassResult'] = [0 if x == 'I' else 1 for x in df_long["passResult"]]

In [70]:
from xgboost import XGBClassifier

params_xgd = {
    'max_depth': 7,
    'objective': 'reg:logistic',
    'learning_rate': 0.05,
    'n_estimators': 10000
    }

X = df_short[['a', 's', 'dis']]
y = df_short['binaryPassResult']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 7)

In [71]:
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier()

In [73]:
y_pred = model.predict(X_test)
preds = [round(value) for value in y_pred]

accuracy_short = metrics.accuracy_score(y_test, preds)
print(accuracy_short)

0.8236331569664903


In [74]:
params_xgd = {
    'max_depth': 7,
    'objective': 'reg:logistic',
    'learning_rate': 0.05,
    'n_estimators': 10000
    }

X = df_long[['a', 's', 'dis']]
y = df_long['binaryPassResult']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 7)

In [75]:
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier()

In [76]:
y_pred = model.predict(X_test)
preds = [round(value) for value in y_pred]

accuracy_deep = metrics.accuracy_score(y_test, preds)
print(accuracy_deep)

0.6408317580340265


In [86]:
accuracy_data = [["Random Forest Classifier", rfc_short_acc, rfc_deep_acc], ["XGBoost", accuracy_short, accuracy_deep]]

col_names = ["Model", "Short Passes", "Long Passes"]

print(tabulate(accuracy_data, headers=col_names))


Model                       Short Passes    Long Passes
------------------------  --------------  -------------
Random Forest Classifier        0.787837       0.573409
XGBoost                         0.823633       0.640832
